Install Packages

In [ ]:
!pip install geemap earthengine-api --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 27.6 MB/s eta 0:00:00


In [ ]:
import ee
import geemap
import os
from datetime import datetime

Authenticate

In [ ]:
ee.Authenticate()   # follow the link, paste token
ee.Initialize(project='ee-raymainh24')

Choose AOI

In [ ]:
Map = geemap.Map()

# Load Nairobi boundary from GEE FeatureCollection
Laikipia = ee.FeatureCollection("FAO/GAUL/2015/level2") \
    .filter(ee.Filter.eq('ADM2_NAME', 'Laikipia'))

Map.centerObject(Laikipia, 10)
Map.addLayer(Laikipia, {}, "Laikipia Boundary")
Map

Map(center=[0.32800176197946224, 36.771931557085175], controls=(WidgetControl(options=['position', 'transparen…

Sentinel-2 collection & cloud masking

In [ ]:
# ---- Fixed Cell 4 ----
start_date = '2024-01-01'
end_date = '2024-12-31'

def mask_s2_sr(image):
    scl = image.select('SCL')
    mask = scl.neq(3).And(scl.neq(8)).And(scl.neq(9)).And(scl.neq(10)).And(scl.neq(11))
    return image.updateMask(mask).copyProperties(image, image.propertyNames())

# ✅ Select only these core bands to ensure homogeneity
BANDS = ['B2', 'B3', 'B4', 'B8', 'B11', 'B12', 'SCL']

col = (
    ee.ImageCollection('COPERNICUS/S2_SR')
    .filterBounds(Laikipia)
    .filterDate(start_date, end_date)
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 80))
    .select(BANDS)      # <---- This line fixes the incompatible bands problem
    .map(mask_s2_sr)
)

print("Filtered image count:", col.size().getInfo())

median2024 = col.median().clip(Laikipia)

Filtered image count: 716


yearly median and longer seasonal composites.

In [ ]:
# Yearly median composite
median2024 = col.median().clip(Laikipia)

# Example: extract bands we need (B2,B3,B4,B8,B11,B12,B3... etc)
bands = ['B2','B3','B4','B8','B11','B12','SCL']
median2024 = median2024.select(['B2','B3','B4','B8','B11','B12'])

Compute indices: NDVI, SAVI, BSI

In [ ]:
# NDVI
ndvi = median2024.normalizedDifference(['B8','B4']).rename('NDVI')

# SAVI: L = 0.5
L = 0.5
savi = median2024.expression(
    '((NIR - RED) / (NIR + RED + L)) * (1 + L)',
    {'NIR': median2024.select('B8'), 'RED': median2024.select('B4'), 'L': L}
).rename('SAVI')

# BSI (example)
bsi = median2024.expression(
    '((SWIR - NIR) / (SWIR + NIR))',  # a simple soil proxy using B11 (SWIR) vs B8
    {'SWIR': median2024.select('B11'), 'NIR': median2024.select('B8')}
).rename('BSI_simple')

# Stack results
indices = ndvi.addBands(savi).addBands(bsi)

Visualize quick maps in Colab

In [ ]:
# Set visualization parameters
rgbVis = {'min': 0, 'max': 3000, 'bands': ['B4','B3','B2']}
ndviVis = {'min': -0.2, 'max': 0.8, 'palette': ['brown','yellow','green']}
bsiVis = {'min': -0.5, 'max': 0.5, 'palette': ['blue','white','red']}

Map.addLayer(median2024, rgbVis, 'Sentinel-2 RGB (median 2024)')
Map.addLayer(ndvi, ndviVis, 'NDVI 2024')
Map.addLayer(bsi, bsiVis, 'BSI 2024')
Map.centerObject(Laikipia, 10)
display(Map)

Map(bottom=131133.0, center=[0.32800176197946224, 36.771931557085175], controls=(WidgetControl(options=['posit…

**Quick stats and zonal summaries**: Compute mean/percentile NDVI across Laikipia and export a small summary table.

In [ ]:
# Compute mean NDVI over Laikipia
mean_ndvi = ndvi.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=Laikipia,
    scale=500,
    maxPixels=1e10
)
print("Mean NDVI:", mean_ndvi.getInfo())

# Histogram sample (NDVI value counts)
ndvi_hist = ndvi.reduceRegion(
    reducer=ee.Reducer.histogram(255),
    geometry=Laikipia,
    scale=500,
    maxPixels=1e10
)
print("NDVI histogram (sample):", ndvi_hist.getInfo())

Mean NDVI: {'NDVI': 0.30457232849407645}
NDVI histogram (sample): {'NDVI': {'bucketMeans': [-0.040954177569524734, -0.037109375, -0.033203125, -0.029296875, -0.025390625, -0.021484375, -0.016558276832249436, -0.013671875, -0.009765625, -0.005859375, -0.001953125, 0.001953125, 0.005859375, 0.009765625, 0.015191111260980241, 0.017578125, 0.021484375, 0.02506910685349347, 0.029545454545454545, 0.033203125, 0.037109375, 0.041015625, 0.044921875, 0.048828125, 0.052734375, 0.056640625, 0.060546875, 0.064453125, 0.068359375, 0.0730727073438071, 0.07776233352209155, 0.080078125, 0.08374289293630144, 0.087890625, 0.091796875, 0.09568441652075393, 0.09980442272688839, 0.10387054580398128, 0.10705466912761728, 0.11251352326000721, 0.11497597771776195, 0.11925311850902626, 0.12244712954510381, 0.12721679032377, 0.13091222126164254, 0.13481044778951415, 0.13908871074269022, 0.1428938060347392, 0.1468063312300002, 0.15051524241853612, 0.15436709613701366, 0.15810963877958636, 0.1621030297624075, 0.1

Export GeoTIFFs to Google Drive

In [ ]:
# Create an export function for convenience
def export_to_drive(image, name, folder='colab_exports', scale=10, crs='EPSG:4326'):
    task = ee.batch.Export.image.toDrive(
        image=image.clip(Laikipia),
        description=name,
        folder=folder,
        fileNamePrefix=name,
        scale=scale,
        region=Laikipia.geometry().bounds().getInfo()['coordinates'],
        crs=crs,
        maxPixels=1e13
    )
    task.start()
    print(f"Export started: {name}. Check your Google Drive -> {folder}")

# Export NDVI
export_to_drive(ndvi, 'Laikipia_NDVI_2024', folder='Laikipia_2024')

# Export BSI
export_to_drive(bsi, 'Laikipia_BSI_2024', folder='Laikipia_2024')

# Export RGB (3-band)
export_to_drive(median2024.select(['B4','B3','B2']), 'Laikipia_RGB_2024', folder='Laikipia_2024')

Export started: Laikipia_NDVI_2024. Check your Google Drive -> Laikipia_2024
Export started: Laikipia_BSI_2024. Check your Google Drive -> Laikipia_2024
Export started: Laikipia_RGB_2024. Check your Google Drive -> Laikipia_2024
